A playground with train dataset

In [1]:
import json

# import train dataset

In [2]:
train_file = "data/KD-DTI/raw/train.json"
with open (train_file, 'r') as f:
    train_dataset = json.load(f)

print(len(train_dataset))


12097


In [3]:
train_pmids_file = "data/KD-DTI/raw/relis_train.pmid"
train_pmids = []

with open (train_pmids_file, 'r') as f:
    for line in f:
        line = line.rstrip()
        train_pmids.append(line)

print(len(train_pmids))

12097


In [4]:
o_train_file = "data/KD-DTI/raw/original_train.json"
with open (o_train_file, 'r') as f:
    original_train_dataset = json.load(f)

print(len(original_train_dataset))


12097


# get test_data and analyze func without syn

In [5]:
def remove_brackets(string):
    if string.startswith("(") and string.endswith(")"):
        return string[1:-1]  # Remove the first and last characters (brackets)
    else:
        return string  # Return the string as is
    # return string

In [6]:
def get_test_data(id, pmid=False, train_dataset=train_dataset, train_pmids=train_pmids):
    test_data = {}
    if pmid:
        test_data['pmid'] = id
    else:
        test_data['pmid'] = train_pmids[id]
    test_data['text'] = train_dataset[test_data['pmid']]['title'].strip() + " " + train_dataset[test_data['pmid']]['abstract']
    test_data['text_no_lower'] = test_data['text']
    test_data['text'] = test_data['text'].lower().strip().replace('  ', ' ')
    # test_data['text] with no spaces
    test_data['text_nospace'] = test_data['text'].replace(' ', '')
    test_data['gold_triples'] = train_dataset[test_data['pmid']]['triples']
    test_data['drugs'] = [item['drug'].lower() for item in test_data['gold_triples']]
    test_data['drugs'] = list(set(test_data['drugs']))
    test_data['drug_segments'] = [item.split(" ") for item in test_data['drugs']]

    # remove brackets from drug segments
    test_data['drug_segments'] = [[remove_brackets(item) for item in segment] for segment in test_data['drug_segments']]

    test_data['targets'] = [item['target'].lower() for item in test_data['gold_triples']]
    test_data['targets'] = list(set(test_data['targets']))
    test_data['target_segments'] = [item.split(" ") for item in test_data['targets']]
    
    # remove brackets from target segments
    test_data['target_segments'] = [[remove_brackets(item) for item in segment] for segment in test_data['target_segments']]
    return test_data

In [7]:
import re

# remove any signs in word.
def remove_signs(word):
    return re.sub(r'[^\w\s]', '', word)

In [8]:
"""
statistical analysis:
num_drug
num_target
drug in text%
target in text%
pmid d and t in text = []
pmid only d in text = []
pmid only t in text = []
pmid no d and t in text = []

-pmid
    -text
    -drugs
    -drug_segments
    -if drug segments in text (no: 0, yes: 1) 
    -target_segments
    -if target segments in text (no: 0, yes: 1) 
"""
def analyze(text):
    ana = {}
    ana["pmid"] = text['pmid']
    ana['drugs'] = text['drugs']
    ana["targets"] = text['targets']
    ana["drug_segments"] = text['drug_segments']
    ana["if drug segments in text"] = []
    ana["target_segments"] = text['target_segments']
    ana["if target segments in text"] = []
    for drug in text['drug_segments']:
        if_setment_in_text = []
        for segment in drug:
            if segment in text['text_nospace']:
                if_setment_in_text.append(1)
            else:
                if_setment_in_text.append(0)
        
        ana["if drug segments in text"].append(if_setment_in_text)


    for target in text['target_segments']:
        if_setment_in_text = []
        for segment in target:
            if segment in text['text_nospace']:
                if_setment_in_text.append(1)
            else:
                if_setment_in_text.append(0)
        
        ana["if target segments in text"].append(if_setment_in_text)

    return ana

# Analyze for all train data without synonims

In [242]:
# analyze for all train data
num_drug = 0
num_drug_in_text = 0

num_target = 0
num_target_in_text = 0
# drug in text%
# target in text%
d_and_t_in_text = []
only_d_in_text = []
only_t_in_text = []
no_d_and_t_in_text = []



for i in range(len(train_pmids)):
    triplet = get_test_data(i)
    num_drug += len(triplet['drugs'])
    num_target += len(triplet['targets'])
    analysis = analyze(triplet)

    if_drug = 1
    if_target = 1
    for if_item_in in analysis["if drug segments in text"]:
        if 0 not in if_item_in:
            num_drug_in_text += 1
        else:
            if_drug = 0
    
    for if_item_in in analysis["if target segments in text"]:
        if 0 not in if_item_in:
            num_target_in_text += 1
        else:
            if_target = 0
    
    if if_drug == 1:
        if if_target == 1:
            d_and_t_in_text.append(triplet['pmid'])
        else:
            only_d_in_text.append(triplet['pmid'])
    elif if_target == 1:
        only_t_in_text.append(triplet['pmid'])
    else:
        no_d_and_t_in_text.append(triplet['pmid'])


In [110]:
print(f"num_drug: {num_drug}\nnum_drug_in_text: {num_drug_in_text}\nnum_target: {num_target}\nnum_target_in_text: {num_target_in_text}")
print(f"drug: {'%.2f' %(num_drug_in_text * 100 / num_drug)}%\ntarget: {'%.2f' %(num_target_in_text * 100 / num_target)}%")

num_drug: 19426
num_drug_in_text: 8663
num_target: 17179
num_target_in_text: 2837
drug: 44.59%
target: 16.51%


In [111]:
print(f"d_and_t_in_text: {len(d_and_t_in_text)}\nonly_d_in_text: {len(only_d_in_text)}\nonly_t_in_text: {len(only_t_in_text)}\nno_d_and_t_in_text: {len(no_d_and_t_in_text)}")
# len(d_and_t_in_text) + len(only_d_in_text) + len(only_t_in_text) + len(no_d_and_t_in_text) == len(train_pmids)

d_and_t_in_text: 1558
only_d_in_text: 5632
only_t_in_text: 653
no_d_and_t_in_text: 4254


# import original drug and target data

In [9]:
import csv
id_to_drug = {}
drug_to_syn = {}

# drug
with open('/home/tian/Projects/BERT-DTI/DBid_to_names/drugbank vocabulary.csv', "r") as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        id = row[0]
        name = row[2].lower()
        syn= row[5].lower().split(" | ")
        id_to_drug[id] = name
        drug_to_syn[name] = syn


In [10]:
# target
with open('/home/tian/Projects/MyReaserch/DBid_to_names/target vocabulary.json') as f:
    target_id_to_name = json.load(f)

print(len(target_id_to_name))
# If the value of target_id_to_name is "This page doesn't exist. What a pain.", then ignore that line.

target_id_to_name = {k: v for k, v in target_id_to_name.items() if v != "This page doesn't exist. What a pain."}
print(f'After filtering: {len(target_id_to_name)}')

10215
After filtering: 5838


In [11]:
# target synonyms

with open('/home/tian/Projects/MyReaserch/DBid_to_names/target vocabulary w syn.json') as f:
    target_name_to_syn = json.load(f)

print(len(target_name_to_syn))
# If the value of target_id_to_name is "This page doesn't exist. What a pain.", then ignore that line.

5205


# Analyze one dataset

In [13]:
id = no_d_and_t_in_text[800]

NameError: name 'no_d_and_t_in_text' is not defined

In [196]:
analyze(get_test_data(id, pmid=True))

{'pmid': '2466833',
 'drugs': ['hyaluronic acid'],
 'targets': ['versican core protein'],
 'drug_segments': [['hyaluronic', 'acid']],
 'if drug segments in text': [[0, 1]],
 'target_segments': [['versican', 'core', 'protein']],
 'if target segments in text': [[0, 0, 1]]}

In [12]:
get_test_data(id, pmid=True)['text']

KeyError: 'DB17386'

In [74]:
# 'pmid': '19053774'
print(f"(v2r): {'(v2r)' in get_test_data(only_t_in_text[5], pmid=True)['text_nospace']}")
print(f"v2r: {remove_signs('(v2r)') in get_test_data(only_t_in_text[5], pmid=True)['text_nospace']}")

(v2r): False
v2r: True


In [197]:
# no_d_and_t_in_text 4254
test = get_test_data(id, pmid=True)
o_test = get_test_data(id, pmid=True, train_dataset=original_train_dataset)

In [198]:
o_test

{'pmid': '2466833',
 'text': 'isolation and partial characterization of a glial hyaluronate-binding protein. a glial hyaluronate-binding protein (ghap) with an isoelectric point of 4.3-4.4 was isolated from human brain white matter. the 60-kda glycoprotein appeared to be quite resistant to proteolysis, and comparison with ghap from a viable glioma removed at surgery showed that the protein isolated from autopsy material was not a degradation product resulting from postmortem autolysis. the protein was localized immunohistochemically with mouse monoclonal and rabbit polyclonal antibodies in cerebral white matter. only small amounts could be found in the gray matter. after enzymatic deglycosylation, an immunoreactive 47-kda polypeptide was obtained. two amino acid sequences of ghap showed a striking similarity (up to 89%) with a highly conserved region of cartilage proteins (bovine nasal cartilage proteoglycan and rat and chicken link protein). however, the amino acid composition and oth

In [199]:
test

{'pmid': '2466833',
 'text': 'isolation and partial characterization of a glial hyaluronate-binding protein. a glial hyaluronate-binding protein (ghap) with an isoelectric point of 4.3-4.4 was isolated from human brain white matter. the 60-kda glycoprotein appeared to be quite resistant to proteolysis, and comparison with ghap from a viable glioma removed at surgery showed that the protein isolated from autopsy material was not a degradation product resulting from postmortem autolysis. the protein was localized immunohistochemically with mouse monoclonal and rabbit polyclonal antibodies in cerebral white matter. only small amounts could be found in the gray matter. after enzymatic deglycosylation, an immunoreactive 47-kda polypeptide was obtained. two amino acid sequences of ghap showed a striking similarity (up to 89%) with a highly conserved region of cartilage proteins (bovine nasal cartilage proteoglycan and rat and chicken link protein). however, the amino acid composition and oth

In [14]:
# def get_syns(id): get synoyms for drug and target
import re

def get_syns(id):
    print(id)
    drugs_syns = []
    targets_syns = [] 
    # test = get_test_data(id, pmid=True)
    o_test = get_test_data(id, pmid=True, train_dataset=original_train_dataset)
    # drug
    for drug in o_test["drugs"]:
        if "##" in drug:
            drug_id = re.search(r'####(.+)', drug).group(1).upper()
            drug_name = id_to_drug[drug_id]
        else:
            drug_name = drug
        # print(id_to_drug[drug_id])
        try:
            if len(drug_to_syn[drug_name][-1]) > 0:
                # print(drug_to_syn[id_to_drug[drug_id]])
                drugs_syns.append(drug_to_syn[drug_name])
            else:
                print(f"Drug: {drug_name} doesn't have syns")
                drugs_syns.append([0])
        except KeyError:
            print(f"Drug: {drug_name} doesn't have syns")
            drugs_syns.append([0])

    # target
    for target in o_test["targets"]:
        if "##" in target:
            target_id = re.search(r'####(.+)', target).group(1).upper()
            target_name = target_id_to_name[target_id]
        else:
            target_name = target
        # print(target_id_to_name[target_id])
        try:
            # print(target_name_to_syn[target_id_to_name[target_id]])
            targets_syns.append(target_name_to_syn[target_name])
        except:
            print(f"Target: {target_name} doesn't have syns")
            targets_syns.append([0])
    
    return drugs_syns, targets_syns

In [15]:
get_test_data(id, pmid=True)
get_syns(id)

KeyError: 'DB17386'

# Analyze func for all train data with synonims

In [16]:
def analyze_w_syn(text):
    ana = {}
    ana["pmid"] = text['pmid']

    ana['drugs'] = text['drugs']
    ana["drug_segments"] = text['drug_segments']
    ana['drug_syns'], ana['target_syns'] = get_syns(text['pmid'])
    ana['drug_syn_segments'] = []
    for syns in ana['drug_syns']:
        syns_segment = []
        if syns == [0]:
            ana['drug_syn_segments'].append([0])

        else:
            for item in syns:
                seg = item.lower().split(" ")
                # remove brackets from seg
                seg = [remove_brackets(item) for item in seg]

                syns_segment.append(seg)
            ana['drug_syn_segments'].append(syns_segment)


    ana["targets"] = text['targets']
    ana["target_segments"] = text['target_segments']
    ana['target_syn_segments'] = []
    for syns in ana['target_syns']:
        # print(f"syns : {syns}")
        syns_segment = []
        if syns == [0] or syns == 0:
            ana['target_syn_segments'].append([0])

        else:
            for item in syns:
                seg = item.lower().split(" ")
                # remove brackets from seg
                seg = [remove_brackets(item) for item in seg]
                syns_segment.append(seg)
            ana['target_syn_segments'].append(syns_segment)


    ana["if drug segments in text"] = []
    ana["if drug syn segments in text"] = []
    ana["if target segments in text"] = []
    ana["if target syn segments in text"] = []

    for drug in text['drug_segments']:
        if_setment_in_text = []
        for segment in drug:
            if segment in text['text_nospace']:
                if_setment_in_text.append(1)
            else:
                if_setment_in_text.append(0)
        
        ana["if drug segments in text"].append(if_setment_in_text)


    for drug_syns in ana['drug_syn_segments']:
        if_syn_in_text = []
        if drug_syns == [0]:
            ana["if drug syn segments in text"].append([0])

        else:
            for syn_segments in drug_syns:
                if_setment_in_text = []
                for segment in syn_segments:
                    # print(f"current segment: {segment}")
                    if segment in text['text_nospace']:
                        if_setment_in_text.append(1)
                    else:
                        if_setment_in_text.append(0)

                # if_syn_in_text.append(if_setment_in_text)
                if 0 in if_setment_in_text:
                    if_syn_in_text.append(0)
                else:
                    if_syn_in_text.append(1)
                # print(f"if_syn_in_text: {if_syn_in_text}")
        
            ana["if drug syn segments in text"].append(if_syn_in_text)


    for target in text['target_segments']:
        if_setment_in_text = []
        for segment in target:
            if segment in text['text_nospace']:
                if_setment_in_text.append(1)
            else:
                if_setment_in_text.append(0)
        
        ana["if target segments in text"].append(if_setment_in_text)

    for target_syns in ana['target_syn_segments']:
        if_syn_in_text = []
        if target_syns == [0]:
            ana["if target syn segments in text"].append([0])

        else:
            for syn_segments in target_syns:
                if_setment_in_text = []
                for segment in syn_segments:
                    if segment in text['text_nospace']:
                        if_setment_in_text.append(1)
                    else:
                        if_setment_in_text.append(0)

                # if_syn_in_text.append(if_setment_in_text)
                if 0 in if_setment_in_text:
                    if_syn_in_text.append(0)
                else:
                    if_syn_in_text.append(1)
        
            ana["if target syn segments in text"].append(if_syn_in_text)

    return ana

# Analyze for all train data with synonims

In [17]:
# analyze for all train data w syns
num_DB_drug = 0
num_DB_drug_in_text = 0
num_DB_drug_syn_in_text = 0

num_DB_target = 0
num_DB_target_in_text = 0
num_DB_target_syn_in_text = 0
# drug in text%
# target in text%
d_and_t_in_text = []
d_and_t_in_syn_text = []
only_d_in_text = []
only_t_in_text = []
no_d_and_t_in_text = []
no_d_and_t_syn_in_text = []



for i in train_pmids:
    data = get_test_data(i, pmid=True)
    pmid = data['pmid']
    original_data = get_test_data(pmid, pmid=True, train_dataset=original_train_dataset)
    # only analyze the data from DrugBank
    # if "##" in original_data['drugs'][0]:
    num_DB_drug += len(data['drugs'])
    num_DB_target += len(data['targets'])
    analysis = analyze_w_syn(data)

    if_drug = 1
    if_drug_syn = 1

    if_target = 1
    if_target_syn = 1
    for if_item_in in analysis["if drug segments in text"]:
        # [1, 0, 0] means the 1st segment is in text, but the 2nd and 3rd segments are not in text, so the span is not in text.
        if 0 not in if_item_in:
            num_DB_drug_in_text += 1
        else:
            if_drug = 0

    for if_item_in in analysis["if drug syn segments in text"]:
        # [1, 0, 0] means the 1st syn is in text, but the 2nd and 3rd syns are not in text.
        if 1 in if_item_in:
            num_DB_drug_syn_in_text += 1
        else:
            if_drug_syn = 0
    
    for if_item_in in analysis["if target segments in text"]:
        if 0 not in if_item_in:
            num_DB_target_in_text += 1
        else:
            if_target = 0
    
    for if_item_in in analysis["if target syn segments in text"]:
        if 1 in if_item_in:
            num_DB_target_syn_in_text += 1
        else:
            if_target_syn = 0
    
    if if_drug == 1:
        if if_target == 1:
            d_and_t_in_text.append(data['pmid'])
        else:
            only_d_in_text.append(data['pmid'])
    elif if_target == 1:
        only_t_in_text.append(data['pmid'])
    else:
        no_d_and_t_in_text.append(data['pmid'])


    if if_drug_syn == 1 or if_drug == 1:
        if if_target_syn == 1 or if_target == 1:
            d_and_t_in_syn_text.append(data['pmid'])
    
    if if_drug == 0 and if_drug_syn == 0:
        if if_target == 0 and if_target_syn == 0:
            no_d_and_t_syn_in_text.append(data['pmid'])
        
    
    # else:
        # pass


7761270
14748653
20439741
Drug: pf-3758309 doesn't have syns
Target: pak-4 protein kinase (pak4) doesn't have syns
16339914
26412054
7511750
Drug: kc-399 doesn't have syns
Target: potassium channel unspecific (kc) doesn't have syns
10908108
Drug: f-12509a doesn't have syns
Target: sphingosine kinase 1 (sphk1) doesn't have syns
17011224
14728043
32180400
12225961
11475940
Drug: valproate doesn't have syns
Target: histone deacetylase (hdac) doesn't have syns
16143263
Drug: abt-100 doesn't have syns
Target: caax farnesyltransferase beta (fntb) doesn't have syns
9831331
22999878
Drug: rocaglamide doesn't have syns
16697227
10583449
12100530
10844127
7929150
17562170
26416534
12184141
Drug: fedotozine doesn't have syns
Target: opioid receptor kappa (oprk1) doesn't have syns
10779697
10522750
12023937
Drug: caffeic acid phenethyl ester doesn't have syns
Target: nuclear factor nf-kappa-b (nfkb) doesn't have syns
18595720
Drug: manzamine a doesn't have syns
Target: glycogen synthase kinase-3 b

In [379]:
get_test_data(0)

{'pmid': '7761270',
 'text': "beta-subunits co-determine the sensitivity of rat neuronal nicotinic receptors to antagonists. we have investigated the effect of 4 ganglionic cholinergic antagonists (hexamethonium, mecamylamine, pentolinium, trimetaphan) on rat alpha 3 beta 2 and alpha 3 beta 4 neuronal nicotinic acetylcholine receptors (nachrs) expressed in xenopus oocytes. current responses were elicited by fast application of acetylcholine on voltage-clamped oocytes (holding potentialvh = -80mv). concentration-inhibition curves were used to get estimates of ic50, the antagonist concentration yielding 50% reduction of the peak current. the kb's of the antagonists were calculated using estimates of the apparent kd of acetylcholine. the order of affinity of the antagonists was similar for both receptor subtypes: mecamylamine approximately pentolinium &gt; hexamethonium &gt; trimetaphan. however, alpha 3 beta 4 neuronal nachrs were 9 to 22 times more sensitive to each of the 4 antagonists

In [19]:
len(train_dataset)

12097

In [18]:
# analyze for all train data w syns
print(f"num_DB_drug (only DB): {num_DB_drug}")
print(f"num_DB_drug_in_text: {num_DB_drug_in_text}")
print(f"num_DB_drug_syn_in_text: {num_DB_drug_syn_in_text}")

print(f"num_DB_target (only DB): {num_DB_target}")
print(f"num_DB_target_in_text: {num_DB_target_in_text}")
print(f"num_DB_target_syn_in_text: {num_DB_target_syn_in_text}")
# drug in text%
# target in text%

# print(f"num_drug: {num_DB_drug}\nnum_drug_in_text: {num_DB_drug_in_text}\nnum_target: {num_DB_target}\nnum_target_in_text: {num_DB_target_in_text}")
print(f"drug: {'%.2f' %(num_DB_drug_in_text * 100 / num_DB_drug)}%\ndrug w syn: {'%.2f' %(num_DB_drug_syn_in_text * 100 / num_DB_drug)}%\
      \ntarget: {'%.2f' %(num_DB_target_in_text * 100 / num_DB_target)}%\ntarget w syn:{'%.2f' %(num_DB_target_syn_in_text * 100 / num_DB_target)}%")
print(f"d_and_t_in_text: {len(d_and_t_in_text)}\nd_and_t_in_syn_text: {len(d_and_t_in_syn_text)}\
      \nno_d_and_t_in_text: {len(no_d_and_t_in_text)}\nno_d_and_t_syn_in_text: {len(no_d_and_t_syn_in_text)}")


# d_and_t_in_text
# d_and_t_in_syn_text
# only_d_in_text
# only_t_in_text
# no_d_and_t_in_text
# no_d_and_t_syn_in_text

num_DB_drug (only DB): 19426
num_DB_drug_in_text: 8670
num_DB_drug_syn_in_text: 6640
num_DB_target (only DB): 17179
num_DB_target_in_text: 3468
num_DB_target_syn_in_text: 3411
drug: 44.63%
drug w syn: 34.18%      
target: 20.19%
target w syn:19.86%
d_and_t_in_text: 1772
d_and_t_in_syn_text: 3354      
no_d_and_t_in_text: 4010
no_d_and_t_syn_in_text: 3436


In [29]:
id = d_and_t_in_text[32]
test = get_test_data(id, pmid=True)
o_test = get_test_data(id, pmid=True, train_dataset=original_train_dataset)
analyze_w_syn(get_test_data(id, pmid=True))
# get_test_data(id, pmid=True)['text']


17068772
Target: dopamine d2 receptor (d2r) doesn't have syns


{'pmid': '17068772',
 'drugs': ['nomifensine'],
 'drug_segments': [['nomifensine']],
 'drug_syns': [['(+-)-nomifensin',
   '(+-)-nomifensine',
   '2-methyl-4-phenyl-1,2,3,4-tetrahydro-isoquinolin-8-ylamine',
   '8-amino-2-methyl-4-phenyl-1,2,3,4-tetrahydroisoquinoline',
   'd,l-nomifensine',
   'nomifensin',
   'nomifensina',
   'nomifensine',
   'nomifensinum']],
 'target_syns': [[0]],
 'drug_syn_segments': [[['(+-)-nomifensin'],
   ['(+-)-nomifensine'],
   ['2-methyl-4-phenyl-1,2,3,4-tetrahydro-isoquinolin-8-ylamine'],
   ['8-amino-2-methyl-4-phenyl-1,2,3,4-tetrahydroisoquinoline'],
   ['d,l-nomifensine'],
   ['nomifensin'],
   ['nomifensina'],
   ['nomifensine'],
   ['nomifensinum']]],
 'targets': ['dopamine d2 receptor (d2r)'],
 'target_segments': [['dopamine', 'd2', 'receptor', 'd2r']],
 'target_syn_segments': [[0]],
 'if drug segments in text': [[1]],
 'if drug syn segments in text': [[0, 0, 0, 0, 0, 1, 0, 1, 0]],
 'if target segments in text': [[1, 1, 1, 1]],
 'if target syn seg

In [26]:
get_syns(id)

1965892


([['etretinate', 'etretinato']],
 [['NR1B1', 'Nuclear receptor subfamily 1 group B member 1', 'RAR-alpha']])

In [30]:
test

{'pmid': '17068772',
 'text': 'pharmacologically induced, subsecond dopamine transients in the caudate-putamen of the anesthetized rat. subsecond dopamine (da) concentration transients have previously been observed in behaving rats. here, we demonstrate for the first time that da transients can be pharmacologically induced in anesthetized rats. coadministration of the d2 receptor antagonist, haloperidol, and the da uptake inhibitor, nomifensine, results in significantly more da transients than either drug alone. the results show that both d2 autoreceptors and the dopamine transporter regulate subsecond da neurotransmission.',
 'text_no_lower': 'Pharmacologically induced, subsecond dopamine transients in the caudate-putamen of the anesthetized rat. Subsecond dopamine (DA) concentration transients have previously been observed in behaving rats. Here, we demonstrate for the first time that DA transients can be pharmacologically induced in anesthetized rats. Coadministration of the D2 rece

In [359]:
o_test

{'pmid': '20374735',
 'text': 'follicle stimulating hormone receptor mutations and reproductive disorders. the follicle stimulating hormone receptor (fshr) plays a critical role in reproductive function. in the males, fsh supports spermatogenesis, whereas in females, fsh is absolutely required for ovarian follicle growth. in females, inactivating mutations in the fshr result in ovarian dysgenesis with amenorrhea and infertility. the few males reported with severe inactivating mutations exhibited varying spermatogenic defects, but not azoospermia. while these findings may potentially suggest that fsh action is not absolutely required for spermatogenesis, it cannot be ruled out that these individuals have some residual fshr activity. gain-of-function mutations in the fshr cause spontaneous ovarian hyperstimulation syndrome in females due to the inappropriate stimulation of the mutant fshr by human choriogonadotropin.',
 'text_no_lower': 'Follicle stimulating hormone receptor mutations an

# find the similar part in the text

In [360]:
# Gestalt similarity

from difflib import SequenceMatcher
def gestalt(a, b):
    return SequenceMatcher(None, a, b).ratio()

def get_similar_words(test_word, text):
    words_set = text.split(" ")
    similar_words_gestalt = []
    for w in words_set:
        if (gestalt(test_word, w) > 0.5) and (w != test_word) and (w not in similar_words_gestalt):
            similar_words_gestalt.append(w)

    print(similar_words_gestalt)
    return(similar_words_gestalt)

In [363]:
gestalt("abc", "abcdddd")

0.6